In [ ]:
import numpy as np 
import pandas as pd 
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


import datatable as dt

from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)
pio.templates.default = "none"
import cv2

import warnings
warnings.filterwarnings('ignore')

* Data loading is slow - could speed up via loading preconverted parquet/feather files

In [ ]:
%%time
path = '/kaggle/input/h-and-m-personalized-fashion-recommendations'

sample_sumbission = dt.fread(path+'/sample_submission.csv').to_pandas()
art = dt.fread(path+'/articles.csv').to_pandas()
cust = dt.fread(path+ '/customers.csv').to_pandas()

In [ ]:
%%time
trans_train = dt.fread(path+'/transactions_train.csv').to_pandas()

In [ ]:
print(trans_train.shape)
print(trans_train.nunique())

In [ ]:
display(trans_train)

In [ ]:
print(art.shape)
print(art.nunique())

In [ ]:
art.drop_duplicates().shape[0]  ## note - would work better by dorp dupes by all cols except for article_id                      

In [ ]:
display(art)

In [ ]:
print(cust.shape)
print(cust.nunique())
display(cust)

## Frequency analysis:
* Do we have customer history?
* Distribution freq of items

 - We see a very long tail of products, and many customers with a small amount of history
     * BUT: Most customers do have some previous purchases! i.e this isn't purely session absed and we CAN use recommenders/CF!

In [ ]:
trans_train["customer_id"].value_counts().describe().round(1)

In [ ]:
trans_train["customer_id"].value_counts().hist();

In [ ]:
trans_train["article_id"].value_counts().describe().round(1)

In [ ]:
trans_train["article_id"].value_counts().hist(bins=30);

In [ ]:
trans_train["article_id"].nunique()

* 20% of articles are very rare (under 9 cases)

In [ ]:
(trans_train["article_id"].value_counts()>9).sum()

In [ ]:
## 10 most popular
art.loc[art["article_id"].isin(trans_train["article_id"].value_counts().sort_values(ascending=False).head(10).index)]